<a href="https://colab.research.google.com/github/RCortez25/Scientific-Machine-Learning/blob/main/Differential_equations/1DWaveEquation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# A
using NeuralPDE, Lux, Optimization, OptimizationOptimJL

# B
import ModelingToolkit: Interval



*   **A**: list of the packages to use in this problem
    * `NeuralPDE` for defining PDEs symbolically and train PINNs
    * `Lux` lightweight NN library to define PINNs architectures
    * `Optimization` interface for setting up loss function from `NeuralPDE` and `Lux`
    * `OptimizationOptimJL` for plugging `Optim.jl`'s algorithms into the `Optimization` interface.
*   **B**: `import A: B` brings only `B` into the scope, in this case, only `Interval` is imported, which is for specifying the domains of the independent variables



In [ ]:
@parameters x t
@variables u(..)
@derivatives Dt' ~ t
@derivatives Dtt'' ~ t
@derivatives Dxx'' ~ x

We start setting up the problem.


*   `@parameters` for specifying the independent variables, in this case, $x$ and $t$
*   `@variables` for specifying the state variable `u(..)`, where `(..)` means that the dependency will be given later
*   `@derivatives` defines the differential operators to be used in the problem. Note that it is the same as

`Dt = Differential(t)`
`Dtt = Differential(t)^2`
`Dxx = Differential(x)^2`

but this macro is commonly used in NeuralPDE problems.

In [ ]:
# A
boundary_conditions = [
    u(0, t) ~ 0,
    u(1, t) ~ 0,
    u(x, 0) ~ x * (1 - x),
    Dt(u(x, 0)) ~ 0
]

# B
domains = [
    x ∈ Interval(0.0, 1.0),
    t ∈ Interval(0.0, 1.0)
]



**A**: Definition of the initial and boundary conditions. We have
* $u(0,t)=0$, at the left boundary $x=0$. This is a Dirichlet BC and together with the next one at $x=1$ indicates that the wave will be fixed at both ends.
* $u(1,t)=0$, at the right boundary $x=1$, Dirchlet BC as well.
* $u(x,0)=x(1-x)$, the initial displacement of the wave at $t=0$. This is the initial shape of the wave and it's a parabola.
* $∂_tu(x,0)=0$, initial velocity at $t=0$, released from rest.


**B**: Definition of the rectangular domain $(x,t)\in[0,1]\times[0,1]$.

In [ ]:
# A
const c = 1.0

# B
equation = [
    Dtt(u(x, t)) ~ (c^2) * Dxx(u(x, t))
]

**A**: Wave speed, in this case $1$ m/s, defined at the global scope with `const` for keeping things stable and fast.

**B**: Definition of the wave equation in symbolic form

$$
∂_{tt}u=c^2\partial_{xx}u
$$

In [ ]:
# A
input_dimension = 2 # x and t
output_dimension = 1

# B
layer_0 = Lux.Dense(input_dimension, 16, Lux.tanh)
layer_1 = Lux.Dense(16, 16, Lux.tanh)
layer_2 = Lux.Dense(16, output_dimension)

# C
NN = Lux.Chain(
    layer_0,
    layer_1,
    layer_2
)

**A** - The input dimension for the neural network will be 2, which is the number of independent variables $x$ and $t$. The output will be the value of the field $u(x,t)$, that is, the network maps

$$\mathbb{R}^2⟶\mathbb{R}:[x,t]→u(x,t)$$

because $u$ is a scalar field.

**B** - Creating of the fully connected layers of the NN, in the form `Lux.Dense(in, out, activation)`.
*   `layer_0` accepts 2 inputs $(x,y)$, has 16 outputs (this is arbitrary and can be changed), and uses `Lux.tanh` which is a tanh activation function.
*   `layer_1` accepts 16 inputs from the previous layer, has 16 outputs (this is arbitrary and can be changed), and uses `Lux.tanh` which is a tanh activation function.
*   `layer_2` accepts 16 inputs from the previous layer and has 1 output, the value of the field $u(x,t)$. In this case the layer is linear (no activation function) because one is predicting any real number (no need for tanh to squeeze the numbers, or ReLU, etc, that constraint the ourput number to a certain set).